In [ ]:
# Código Anderson

from google.colab import files

#Carregar o arquivo em excel para o colab
uploaded = files.upload()

# Itera sobre as chaves do dicionário 'uploaded'
for filename in uploaded.keys():

    # Imprime uma mensagem informando o sucesso do carregamento do arquivo e seu tamanho em bytes
    print(f'Arquivo "{filename}" carregado com sucesso. Tamanho: {len(uploaded[filename])} bytes.')


Saving Base Padrão GM.xlsx to Base Padrão GM.xlsx
Arquivo "Base Padrão GM.xlsx" carregado com sucesso. Tamanho: 23919145 bytes.


In [ ]:
import os
import pandas as pd


# Lista de nomes dos arquivos
file_names = [
     'Base Padrão GM.xlsx',


]

# Diretório onde os arquivos estão armazenados (atualize para o caminho correto, se necessário)
directory = '/content'

# Função para listar as planilhas de um arquivo Excel
def list_sheets(file_path):
    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
        print(f"Nomes das planilhas no arquivo '{os.path.basename(file_path)}':", sheet_names)
    except Exception as e:
        print(f"Erro ao ler o arquivo '{os.path.basename(file_path)}':", e)

# Verifica se os arquivos existem e lista os nomes das planilhas
for file_name in file_names:
    file_path = os.path.join(directory, file_name)
    if os.path.isfile(file_path):
        list_sheets(file_path)
    else:
        print(f"Arquivo '{file_name}' não encontrado no diretório '{directory}'.")


Nomes das planilhas no arquivo 'Base Padrão GM.xlsx': ['Din', 'alterar ', 'LISTA DE PDMS', 'Zdmat0425', 'Tabela']


In [ ]:

def carregar_dados(arquivo):
   import pandas as pd
   zdmat_df = pd.read_excel(arquivo, sheet_name='Zdmat0425')
   lista_pdms_df = pd.read_excel(arquivo, sheet_name='LISTA DE PDMS')
   tabelas_df = pd.read_excel(arquivo, sheet_name='Tabela')
   return zdmat_df, lista_pdms_df, tabelas_df

# Chamar a função e capturar os três DataFrames
zdmat_df, lista_pdms_df, tabela_df = carregar_dados('Base Padrão GM.xlsx')

# Agora você pode limpar os nomes das colunas
zdmat_df.columns = zdmat_df.columns.str.strip()
lista_pdms_df.columns = lista_pdms_df.columns.str.strip()
tabela_df.columns = tabela_df.columns.str.strip()



In [ ]:
print(lista_pdms_df.columns)
print(zdmat_df.columns)
print(tabela_df.columns)


Index(['Classe', 'Denominação de classe', 'REGRA ATUAL', 'LEAD BUYER'], dtype='object')
Index(['Centro', 'Código', 'Material', 'Texto Longo', 'Grupo de mercadorias',
       'Lead Buyer GM', 'Classe'],
      dtype='object')
Index(['Grupo de Mercadoria', 'Descrição', 'Fabricante/Ref. Comercial',
       'Grupo de Mercadoria.1', 'Descrição.1', 'obs'],
      dtype='object')


In [ ]:
# Converter explicitamente para string e deixar em minúsculas
import pandas as pd

file_name = 'Base Padrão GM.xlsx'

# Verifique se o arquivo pode ser carregado
try:
    df = pd.read_excel(file_name, sheet_name='Zdmat0425')
    print("Arquivo carregado com sucesso!")
    print("Colunas disponíveis:", df.columns.tolist())
except Exception as e:
    print("Erro ao carregar o arquivo:", e)


zdmat_df['Material_lower'] = zdmat_df['Material'].astype(str).str.lower()
zdmat_df['Texto Longo_lower'] = zdmat_df['Texto Longo'].astype(str).str.lower()


print("Colunas disponíveis:", df.columns.tolist())
df.columns = df.columns.str.strip()  # Remove espaços extras nos nomes das colunas
# Confere se tem valores vazios (apenas espaços ou strings vazias)
print((zdmat_df['Texto Longo'].str.strip() == '').sum())

# Mostra os primeiros valores suspeitos
print(zdmat_df[zdmat_df['Texto Longo'].str.strip() == ''].head(10))

zdmat_df['Texto Longo'] = zdmat_df['Texto Longo'].astype(str)
from sklearn.feature_extraction.text import TfidfVectorizer

texts_teste = zdmat_df['Texto Longo'].head(100).tolist()

vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(texts_teste)


Arquivo carregado com sucesso!
Colunas disponíveis: ['Centro', 'Código', 'Material', 'Texto Longo', 'Grupo de mercadorias', 'Lead Buyer GM', 'Classe']
Colunas disponíveis: ['Centro', 'Código', 'Material', 'Texto Longo', 'Grupo de mercadorias', 'Lead Buyer GM', 'Classe']
0
Empty DataFrame
Columns: [Centro, Código, Material, Texto Longo, Grupo de mercadorias, Lead Buyer GM, Classe, Material_lower, Texto Longo_lower]
Index: []


In [ ]:
# Célula que verifica se pode ser material de desenho
import pandas as pd
import re
import string


# === Funções de limpeza e análise ===
def limpar_texto(texto):
    texto = texto.lower()
    texto = re.sub(f"[{string.punctuation}]", " ", texto)
    texto = re.sub(r'\s+', ' ', texto)
    return texto.strip()

def extrair_palavras_chave(texto, stopwords=[]):
    palavras = limpar_texto(texto).split()
    return [p for p in palavras if p not in stopwords and len(p) > 3]

def classificar_por_palavras_chave(df_definicoes, textos_longs, limiar=2):
    categorias = []
    grupos = []

    stopwords = ['com', 'para', 'como', 'que', 'por', 'das', 'dos', 'não', 'em', 'uma', 'item', 'itens', 'será', 'caso', 'etc']


    for texto in textos_longs:
        texto_limpo = limpar_texto(texto)
        encontrado = False

        for _, row in df_definicoes.iterrows():
            palavras_def = extrair_palavras_chave(row['definicao'], stopwords)
            contagem = sum(p in texto_limpo for p in palavras_def)

            if contagem >= limiar:
                categorias.append(row['categoria'])
                grupos.append(row['grupo_mercadoria'])
                encontrado = True
                break  # parar no primeiro match

        if not encontrado:
            categorias.append('Não identificado')
            grupos.append('Não identificado')

    return categorias, grupos


import pandas as pd

# === Base com todas as definições ===
def carregar_definicoes_mmcd():
    dados = [
        {'grupo_mercadoria': '0432EGEG', 'categoria': 'MMCD-Engrenagens e afins',
         'definicao': 'Materiais conforme desenho que possuem "dentes", com características específicas de fabricação/produção, materiais ferrosos ou não ferrosos (exceto plásticos, polímeros, nylon, etc). Exemplos: Engrenagens, pinhão, coroa, rodas dentadas, cremalheira, redutores e outros em que houver "dentes".'},

        {'grupo_mercadoria': '0432AIAI', 'categoria': 'MMCD-Aço inox',
         'definicao': 'Itens que possuem como matéria prima principal o Aço Inox (AISI). Não considerar fundição em inox (nesse caso utilizar Fundição). Engloba tanto usinagem quanto caldeiraria.'},

        {'grupo_mercadoria': '0432MOMO', 'categoria': 'MMCD-Itens morgoil',
         'definicao': 'Item específico – Camisas Morgoil.'},

        {'grupo_mercadoria': '0432CLCL', 'categoria': 'MMCD-Carga líquida',
         'definicao': 'Itens utilizados em equipamentos de movimentação de carga líquida, já homologados pela planta (comissão de carga líquida, segurança, engenharia, suprimentos, etc).'},

        {'grupo_mercadoria': '0432PDPD', 'categoria': 'MMCD-Chapas de desgaste',
         'definicao': 'Engloba chapas em materiais cerâmicos e revestidas com solda (DIN 8555). Considerar usinagem/caldeiraria as chapas em aço baixo e médio carbono e também chapas em aço carbono sem revestimentos. Chapas em cobre/bronze utilizar MMCD-Não Ferrosos. Chapas em celeron, teflon e plásticos de engenharia utilizar MMCD-Isolantes Plásticos e Elastômeros. As chapas de desgaste que não possuem desenhos (compra via referência), utilizar Chap Desg Conf Norma.'},

        {'grupo_mercadoria': '0432LQLQ', 'categoria': 'MMCD-Chuveiros LTQ',
         'definicao': 'Itens específicos do LTQ (Chuveiros). Normalmente em Alumínio Naval, Inox ou Aço Carbono.'},

        {'grupo_mercadoria': '0432OROR', 'categoria': 'MMCD-Correntes',
         'definicao': 'Correntes fabricadas conforme desenho cujo processo de fabricação é Forjaria.'},

        {'grupo_mercadoria': '0432EMEM', 'categoria': 'MMCD-Elementos de refrigeração',
         'definicao': 'Materiais utilizados para refrigeração de fornos, despoeiramento primário e partes acessórias nas Aciarias. Exemplos: Painéis refrigerados, túneis, dutos, etc.'},

        {'grupo_mercadoria': '0432GUGU', 'categoria': 'MMCD-Ferramentas de furo gusa',
         'definicao': 'Materiais específicos para furo de panelas.'},

        {'grupo_mercadoria': '0432EFEF', 'categoria': 'MMCD-Forjados',
         'definicao': 'Materiais que são forjados. Inclui rodas de ponte rolantes, rolos, roldanas, acoplamentos, etc. Possuem na descrição a informação de processo de fabricação forjado e/ou material base de fabricação forjado.'},

        {'grupo_mercadoria': '0432UNUN', 'categoria': 'MMCD-Fundidos',
         'definicao': 'Materiais que são fundidos, inclusive itens em Inox (exceto carcaça de bombas). Inclui guias, insertos de laminação, mancais, etc. Possuem na descrição a informação de processo de fabricação fundido e/ou material base de fabricação fundido.'},

        {'grupo_mercadoria': '0432IPIP', 'categoria': 'MMCD-Isolantes plásticos / elastômeros',
         'definicao': 'Materiais fabricados conforme desenho cuja matéria prima principal seja celeron, borrachas, nylon, poliuretanos, plásticos de engenharia, termoplásticos, polímeros em geral (exceto mangueiras, gaxetas, juntas, etc).'},

        {'grupo_mercadoria': '0432LVLV', 'categoria': 'MMCD-Lâminas e Navalhas',
         'definicao': 'Materiais fabricados conforme desenho, em aço especial, cuja função é corte. Exemplos: Facas, lâminas, navalhas, buchas/rolos de corte em aços ferramenta, tesouras, etc.'},

        {'grupo_mercadoria': '0432EXEX', 'categoria': 'MMCD-Mancais MLC',
         'definicao': 'Materiais (mancais) fabricados conforme desenho, específicos para o MLC.'},

        {'grupo_mercadoria': '0432SLSL', 'categoria': 'MMCD-Materiais Selagem MLC',
         'definicao': 'Materiais fabricados conforme desenho, específicos para o MLC.'},

        {'grupo_mercadoria': '0432MDMD', 'categoria': 'MMCD-Metal duro',
         'definicao': 'Materiais em metal duro, fornecidos por fornecedores específicos como Durit, Atomat, Brassinter, Hardy, Saar HartMetall, entre outros. Também materiais cuja matéria prima seja Carboneto de Tungstênio, além de roletes de laminação (não se aplica para Chapas de Desgaste). Exemplos: Fieiras, roletes de laminação, etc.'},

        {'grupo_mercadoria': '0432MLML', 'categoria': 'MMCD-Molas',
         'definicao': 'Molas em geral conforme desenho de fabricação, exceto aquelas que serão fabricadas conforme norma (usar GM específico Mola conforme Norma).'},

        {'grupo_mercadoria': '0432USUS', 'categoria': 'MMCD-Moldes Gusa',
         'definicao': 'Materiais específicos – Moldes para fabricação de gusa.'},

        {'grupo_mercadoria': '0432NFNF', 'categoria': 'MMCD-Não ferrosos',
         'definicao': 'Materiais que possuem como matéria prima cobre, bronze, latão, alumínio, chumbo, zinco, etc. Não utilizar as categorias de Usinagem e Caldeiraria para itens com esse tipo de material base.'},

        {'grupo_mercadoria': '0432RMRM', 'categoria': 'MMCD-Rolos bimetálicos',
         'definicao': 'Subcategoria específica – Rolos Bimetálicos.'},

        {'grupo_mercadoria': '0432TPTP', 'categoria': 'MMCD-Telas de peneira',
         'definicao': 'Subcategoria específica – Peneiras. Para itens conforme Norma, utilizar GM específico.'},

        {'grupo_mercadoria': '0432UCUC', 'categoria': 'MMCD-Usinagem lev/med',
         'definicao': 'Materiais em Aço Carbono que apresentam em sua maior parte serviços de usinagem ou caldeiraria, com peso menor que 1.000 Kg. Exemplos: Anéis, buchas, espaçadores, suportes, chavetas, pinos, conjuntos, dispositivos, tampas, bases, etc.'},

        {'grupo_mercadoria': '0432UPUP', 'categoria': 'MMCD-Usinag. Pesada',
         'definicao': 'Materiais em Aço Carbono que apresentam em sua maior parte serviços de usinagem ou caldeiraria, com peso maior que 1.000 Kg. Exemplos: Estruturas metálicas, bases, etc.'},

        {'grupo_mercadoria': '0432UEUE', 'categoria': 'MMCD-Itens Laminação Etage',
         'definicao': 'Materiais específicos do fornecedor Etage, mas que podem ser substituídos por outros fornecedores como Açokorte, Danieli, Metalfund, Atomat, etc.'},

        {'grupo_mercadoria': '0432AOAO', 'categoria': 'MMCD-Moldes Alto Forno',
         'definicao': 'Materiais específicos – Moldes para Alto Forno.'},

        {'grupo_mercadoria': '0432ITIT', 'categoria': 'MMCD-Itens Morgan',
         'definicao': 'Materiais específicos do fornecedor Morgan/Primetals, que podem ser adquiridos com outros fornecedores como SMC, Etage, SMS.'},

        {'grupo_mercadoria': '0432KHKH', 'categoria': 'MMCD-Itens Laminação e Aciaria',
         'definicao': 'Materiais críticos e específicos para laminação e aciaria não enquadrados nas subcategorias acima. Normalmente possuem fabricação exclusiva ou com número limitado de fornecedores (máximo 3) com know-how técnico. Deve-se evitar utilizar essa categoria por ser um grupo muito específico. Exemplos de fornecedores: Bombardieri, Gustav, Castellini, SMS, Danieli, Federal, Mogul, Açokorte, Infasul, CSN Carl Schreider, KME Germany, Josua Corts, Etage.'}
    ]


    return pd.DataFrame(dados)



# === INÍCIO DO FLUXO ===
df_definicoes = carregar_definicoes_mmcd()
zdmat_df = pd.read_excel('Base Padrão GM.xlsx', sheet_name='Zdmat0425')
zdmat_df.columns = zdmat_df.columns.str.strip()
zdmat_df['Texto Longo'] = zdmat_df['Texto Longo'].fillna('').astype(str)

# Classificação
categorias, grupos = classificar_por_palavras_chave(df_definicoes, zdmat_df['Texto Longo'])

zdmat_df['Categoria MMCD'] = categorias
zdmat_df['Grupo de Mercadoria'] = grupos

# Resultado final
resultado = zdmat_df[['Material', 'Texto Longo', 'Categoria MMCD', 'Grupo de Mercadoria']]
resultado.to_excel('Classificacao_MMCD_TabelaManual.xlsx', index=False)
print(resultado.head(10))


                                   Material  \
0  CABO ELE PREP CONDUMIG 1X2,5MM 450V/750V   
1                      ROTOR METSO 505279-1   
2         REVESTIM. TRASEIRO METSO 505280-1   
3  EIXO 1)RTBTZ204I182 POS1E2 ; RTBTZ204I18   
4             CAIXA DO MANCAL WEIR E004MG01   
5        POLIA MOVIDA METALMAG 14WNPLMV040E   
6      ROLO ESTICADOR METALMAG 14WNROES040E   
7         REVESTIMENTO METSO AX01-100163164   
8     TRANSFORMADOR METALMAG 14WN040ETRAN02   
9     TRANSFORMADOR METALMAG 14WN055ETRAN02   

                                         Texto Longo  \
0  CABO ELÉTRICO PREPARADO ; TIPO: FLEXÍVEL ; NÚM...   
1  NOME: ROTOR ; APLICAÇÃO: BOMBA HM50 FHC-S W8WF...   
2  NOME: REVESTIM. TRASEIRO ; APLICAÇÃO: BOMBA HM...   
3  FABRICAÇÃO - JUIZ DE FORA ; NOME: EIXO ; MATER...   
4  NOME: CAIXA DO MANCAL ; APLICAÇÃO: BOMBA WARMA...   
5  NOME: POLIA MOVIDA ; APLICAÇÃO: EXTRATOR DE SU...   
6  NOME: ROLO ESTICADOR ; APLICAÇÃO: EXTRATOR DE ...   
7  PENEIRA - ACESSÓRIOS E PEÇAS ; 

In [ ]:
from google.colab import files

# Função para fazer o download do arquivo gerado
def fazer_download(output_file='Classificacao_MMCD_TabelaManual.xlsx'):
    files.download(output_file)

# Fazer o download do arquivo gerado
fazer_download(output_file='Classificacao_MMCD_TabelaManual.xlsx')







<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

# Leitura do arquivo
arquivo = 'Base Padrão GM.xlsx'
tabela_df = pd.read_excel(arquivo, sheet_name='Tabela')
zdmat_df = pd.read_excel(arquivo, sheet_name='Zdmat0425')
lista_pdms_df = pd.read_excel(arquivo, sheet_name='LISTA DE PDMS')

# Limpeza da base de palavras
tabela_df = tabela_df[['Grupo de Mercadoria', 'Fabricante/Ref. Comercial']].dropna()
tabela_df['Fabricante/Ref. Comercial'] = tabela_df['Fabricante/Ref. Comercial'].astype(str).str.strip().str.upper()

# Pré-processamento do zdmat_df
zdmat_df['Material'] = zdmat_df['Material'].astype(str).str.upper()
zdmat_df['Texto Longo'] = zdmat_df['Texto Longo'].astype(str).str.upper()

# Inicialização das colunas de resultado
zdmat_df['Palavra Encontrada'] = np.nan
zdmat_df['GM Palavra'] = np.nan
zdmat_df['Classe (palavra)'] = np.nan
zdmat_df['REGRA ATUAL'] = np.nan

# Palavra por palavra
for idx, row in tabela_df.iterrows():
    palavra = row['Fabricante/Ref. Comercial']
    grupo_mercadoria = row['Grupo de Mercadoria']

    mask = zdmat_df['GM Palavra'].isna() & (
        zdmat_df['Material'].str.contains(palavra, na=False) |
        zdmat_df['Texto Longo'].str.contains(palavra, na=False)
    )

    zdmat_df.loc[mask, 'Palavra Encontrada'] = palavra
    zdmat_df.loc[mask, 'GM Palavra'] = grupo_mercadoria

    classe_match = lista_pdms_df[lista_pdms_df['Classe'].astype(str).str.upper().str.contains(palavra, na=False)]

    if not classe_match.empty:
        classe_valor = classe_match.iloc[0]['Classe']
        regra_valor = classe_match.iloc[0]['REGRA ATUAL']
        zdmat_df.loc[mask, 'Classe (palavra)'] = classe_valor
        zdmat_df.loc[mask, 'REGRA ATUAL'] = regra_valor

# Seleciona colunas na ordem pedida
colunas_resultado = [
    'Centro', 'Código', 'Material', 'Classe', 'Grupo de mercadorias',
    'Lead Buyer GM', 'Palavra Encontrada', 'GM Palavra', 'Classe (palavra)', 'REGRA ATUAL'
]

resultado = zdmat_df[colunas_resultado]

# Mostrar totais
total_linhas = len(zdmat_df)
linhas_encontradas = resultado['GM Palavra'].notna().sum()

print(f"\nTotal de linhas analisadas: {total_linhas}")
print(f"Linhas com palavras encontradas: {linhas_encontradas}")

print("\nExemplos encontrados:\n")
print(resultado[resultado['GM Palavra'].notna()].head(10))

# Salvar resultado principal
resultado.to_excel('Palavras_Encontradas.xlsx', index=False)


# VERIFICAÇÃO DA COLUNA STATUS

def verificar_status(row):
    grupo_mercadoria = str(row['Grupo de mercadorias']).strip()
    gm_palavra = str(row['GM Palavra']).strip()
    regra_atual = str(row['REGRA ATUAL']).strip()

    if pd.isna(gm_palavra) or gm_palavra == "":
        return ""

    if pd.isna(regra_atual) or regra_atual == "":
        if grupo_mercadoria == gm_palavra:
            return "OK"
        else:
            return f"Modificar para {gm_palavra}"

    if grupo_mercadoria == gm_palavra:
        regras = [r.strip().replace('*', '') for r in regra_atual.split(';')]
        if any(grupo_mercadoria.startswith(regra) for regra in regras):
            return "OK"

    return f"Modificar para {gm_palavra}"

# Aplica apenas para linhas com Palavra Encontrada
resultado_com_status = resultado.copy()
mascara_palavra = resultado_com_status['Palavra Encontrada'].notna()
resultado_com_status.loc[mascara_palavra, 'Status'] = resultado_com_status[mascara_palavra].apply(verificar_status, axis=1)

# Salvar planilha com a coluna Status
resultado_com_status.to_excel('Regras_status.xlsx', index=False)

print("\n✅ Verificação de Status concluída.")
print("Arquivo salvo como: Regras_status.xlsx")
print(f"Total de linhas com palavras analisadas para status: {mascara_palavra.sum()}")

print("\nExemplos com Status:\n")
print(
    resultado_com_status[['Material', 'Grupo de mercadorias', 'GM Palavra', 'REGRA ATUAL', 'Status']]
    .dropna(subset=['Status'])
    .query("Status != ''")
    .head(10)
)



Total de linhas analisadas: 172093
Linhas com palavras encontradas: 79321

Exemplos encontrados:

   Centro      Código                                  Material  \
1    SA20    19169901                      ROTOR METSO 505279-1   
2    SA20    19169902         REVESTIM. TRASEIRO METSO 505280-1   
3    4000    19225800  EIXO 1)RTBTZ204I182 POS1E2 ; RTBTZ204I18   
7    SA20    19250602         REVESTIMENTO METSO AX01-100163164   
14   9840     7397000         ARRUELA M16 PAUL WURTH 0576777/14   
22   AP00  2000004402             MIDIAS FISICAS ITEM 4 SIEMENS   
23   4200    19159204   CAIXA DE TRANSPORTE DE-4200-4270MEC0671   
24   4000    19150404       BASE DE MANCAL RCETM001I049 POS1A10   
25   4000    19151200                    PINO (R)CL-AM001-I-055   
26   4000    19152201  ESPAÇADOR ROLETE (R)TB-MT242-I-036 POS G   

               Classe Grupo de mercadorias  Lead Buyer GM Palavra Encontrada  \
1      BOMBA_PECA_AMB             0425KMKM            NaN              METSO   
2  

In [ ]:
from google.colab import files

# Função para fazer o download do arquivo gerado
def fazer_download(output_file='Regras_status.xlsx'):
    files.download(output_file)

# Fazer o download do arquivo gerado
fazer_download(output_file='Regras_status.xlsx')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files

# Função para fazer o download do arquivo gerado
def fazer_download(output_file='Resultados_Agrupados_Com_Moda.xlsx'):
    files.download(output_file)

# Fazer o download do arquivo gerado
fazer_download(output_file='Resultados_Agrupados_Com_Moda.xlsx')



FileNotFoundError: Cannot find file: Resultados_Agrupados_Com_Moda.xlsx

In [ ]:
# Função principal para processar o arquivo
def process_excel_file(file_name):
    zdmat_df, lista_pdms_df = carregar_dados(file_name)
    if zdmat_df is not None and lista_pdms_df is not None:
        resultados_df = buscar_palavras_e_classes(zdmat_df, lista_pdms_df, palavras_chave)
        print(f"Total de linhas analisadas: {len(zdmat_df)}")
        print(f"Total de linhas com resultados: {len(resultados_df)}")
        print(resultados_df.head(10))
        resultados_df.to_excel('Resultados_Processados.xlsx', index=False)
        print("Arquivo 'Resultados_Processados.xlsx' gerado com sucesso!")

# Execução
file_name = 'Base Padrão GM.xlsx'
process_excel_file(file_name)


In [ ]:

import os

# Lista todos os arquivos no diretório atual
files_in_directory = os.listdir()

# Imprime os nomes dos arquivos
for file in files_in_directory:

   print(file)


.config
Base Padrão GM.xlsx
MMCD - Descr Grupos de Mercadoria (1).pdf
sample_data


In [ ]:
# Nova função com tratamento especial quando "REGRA ATUAL" está vazia
def verificar_status(row):
    grupo_mercadoria = str(row['GMZDMat	']).strip()
    gm_palavra = str(row['GM (Moda)	']).strip()
    regra_atual = str(row['Regra']).strip()

    if pd.isna(gm_palavra) or gm_palavra == "":
        return ""  # sem GM Palavra = não avalia

    if pd.isna(regra_atual) or regra_atual == "":
        # Sem regra: comparar direto GM Palavra com Grupo de Mercadorias
        if grupo_mercadoria == gm_palavra:
            return "OK"
        else:
            return f"Modificar para {GM (Moda)}"

    # Se há regra, aplicar verificação
    if grupo_mercadoria == GM (Moda)	:
        regras = [r.strip().replace('*', '') for r in regra_atual.split(';')]
        if any(grupo_mercadoria.startswith(regra) for regra in regras):
            return "OK"

    return f"Modificar para {GM (Moda)}"


# Aplicar somente onde houve palavra encontrada
resultado_com_status = resultado.copy()
mascara_palavra = resultado_com_status['Palavra Encontrada'].notna()
resultado_com_status.loc[mascara_palavra, 'Status'] = resultado_com_status[mascara_palavra].apply(verificar_status, axis=1)

# Salvar nova planilha com a coluna 'Status'
resultado_com_status.to_excel('Regras_status.xlsx', index=False)

print("\n✅ Verificação de Status concluída.")
print("Arquivo salvo como: Regras_status.xlsx")
print(f"Total de linhas com palavras analisadas para status: {mascara_palavra.sum()}")
print("\nExemplos com Status:\n")
